In [117]:
submissions = 'submissions'

In [118]:
files = !ls {submissions}/submission*

In [119]:
import pandas as pd

In [120]:
merged = pd.read_csv('submissions/submission014.csv')
passengerIds = merged['PassengerId']
merged.drop(['Survived', 'PassengerId'], inplace=True, axis=1)

In [121]:
for idx, file in enumerate(files):
    df = pd.read_csv(f'{file}')
    df.columns = ['PassengerId', f'sub_{idx}']
    df.drop('PassengerId', inplace=True, axis=1)
    merged = merged.merge(df, left_index=True, right_index=True)

In [122]:
merged.head(1)

,sub_0,sub_1,sub_2,sub_3,sub_4,sub_5,sub_6,sub_7,sub_8,sub_9,...,sub_16,sub_17,sub_18,sub_19,sub_20,sub_21,sub_22,sub_23,sub_24,sub_25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
merged.mean()

sub_0     0.354067
sub_1     0.387560
sub_2     0.385167
sub_3     0.358852
sub_4     0.370813
sub_5     0.547847
sub_6     0.624402
sub_7     0.382775
sub_8     0.389952
sub_9     0.387560
sub_10    0.349282
sub_11    0.354067
sub_12    0.387560
sub_13    0.358852
sub_14    0.294258
sub_15    0.011962
sub_16    0.389952
sub_17    0.377990
sub_18    0.550239
sub_19    0.433014
sub_20    0.399522
sub_21    0.531100
sub_22    0.392344
sub_23    0.279904
sub_24    0.322967
sub_25    0.399522
dtype: float64

In [124]:
%matplotlib inline
merged.mean(axis=1).mean()

0.3854435038645568

In [125]:
# for the future self: I'm thinking mean survival across all models most likely is what a good model survival should predict
(merged.mean(axis=1) > 0.47).astype(int).mean()

0.38995215311004783

In [126]:
# LB: 0.7846, best I had so far
df_submit = pd.DataFrame({'PassengerId': passengerIds, 'Survived': (merged.mean(axis=1) > 0.47).astype(int)})
df_submit.to_csv(f'{submissions}/_blend050.csv', index=False)

In [127]:
!open {submissions}

In [128]:
!open https://www.kaggle.com/c/titanic/submit

In [129]:
survivors_by_model = merged.mean().sort_values()
survivors_by_model

sub_15    0.011962
sub_23    0.279904
sub_14    0.294258
sub_24    0.322967
sub_10    0.349282
sub_0     0.354067
sub_11    0.354067
sub_3     0.358852
sub_13    0.358852
sub_4     0.370813
sub_17    0.377990
sub_7     0.382775
sub_2     0.385167
sub_12    0.387560
sub_1     0.387560
sub_9     0.387560
sub_8     0.389952
sub_16    0.389952
sub_22    0.392344
sub_20    0.399522
sub_25    0.399522
sub_19    0.433014
sub_21    0.531100
sub_5     0.547847
sub_18    0.550239
sub_6     0.624402
dtype: float64

In [130]:
merged1 = merged.copy()

In [131]:
pd.cut(survivors_by_model, bins = 10)

sub_15    (0.0113, 0.0732]
sub_23      (0.257, 0.318]
sub_14      (0.257, 0.318]
sub_24      (0.318, 0.379]
sub_10      (0.318, 0.379]
sub_0       (0.318, 0.379]
sub_11      (0.318, 0.379]
sub_3       (0.318, 0.379]
sub_13      (0.318, 0.379]
sub_4       (0.318, 0.379]
sub_17      (0.318, 0.379]
sub_7       (0.379, 0.441]
sub_2       (0.379, 0.441]
sub_12      (0.379, 0.441]
sub_1       (0.379, 0.441]
sub_9       (0.379, 0.441]
sub_8       (0.379, 0.441]
sub_16      (0.379, 0.441]
sub_22      (0.379, 0.441]
sub_20      (0.379, 0.441]
sub_25      (0.379, 0.441]
sub_19      (0.379, 0.441]
sub_21      (0.502, 0.563]
sub_5       (0.502, 0.563]
sub_18      (0.502, 0.563]
sub_6       (0.563, 0.624]
dtype: category
Categories (10, interval[float64]): [(0.0113, 0.0732] < (0.0732, 0.134] < (0.134, 0.196] < (0.196, 0.257] ... (0.379, 0.441] < (0.441, 0.502] < (0.502, 0.563] < (0.563, 0.624]]

In [132]:
# sub_15    0.011962 that's a very pessimistic thing
merged1.drop('sub_15', axis=1, inplace=True)

In [133]:
(merged1.mean(axis=1) > 0.50).astype(int).mean()

0.38995215311004783

In [134]:
# LB: 0.78468, didn't change vs prev, which can be explained by the impact 1 out of 22 models can have when averaging
df_submit = pd.DataFrame({'PassengerId': passengerIds, 'Survived': (merged1.mean(axis=1) > 0.50).astype(int)})
df_submit.to_csv(f'{submissions}/_blend051.csv', index=False)

In [135]:
# assuming survival in test is rougly what is in train - good models should float around that
df_train = pd.read_csv(f'raw/train.csv')
df_train['Survived'].mean()

0.3838383838383838

In [136]:
survivors_rate_diff = (survivors_by_model - 0.3838383838383838).abs().sort_values()
survivors_rate_diff

sub_7     0.001063
sub_2     0.001329
sub_9     0.003721
sub_1     0.003721
sub_12    0.003721
sub_17    0.005848
sub_8     0.006114
sub_16    0.006114
sub_22    0.008506
sub_4     0.013025
sub_25    0.015683
sub_20    0.015683
sub_3     0.024987
sub_13    0.024987
sub_11    0.029771
sub_0     0.029771
sub_10    0.034556
sub_19    0.049176
sub_24    0.060872
sub_14    0.089580
sub_23    0.103934
sub_21    0.147262
sub_5     0.164009
sub_18    0.166401
sub_6     0.240564
sub_15    0.371877
dtype: float64

In [137]:
merged2 = merged[survivors_by_model[survivors_rate_diff < 0.02].index.values]

In [138]:
# LB: 0.7751, mb averaging alike models is not the best idea
(merged2.mean(axis=1) > 0.42).astype(int).mean()

0.3875598086124402

In [139]:
df_submit = pd.DataFrame({'PassengerId': passengerIds, 'Survived': (merged2.mean(axis=1) > 0.42).astype(int)})
df_submit.to_csv(f'{submissions}/_blend052.csv', index=False)

In [140]:
merged3 = merged[survivors_by_model[survivors_rate_diff >= 0.02].index.values]

In [141]:
(merged3.mean(axis=1) > 0.49).astype(int).mean()

0.4019138755980861

In [142]:
# LB: 0.7272, I wonder if the above 0.78 is a result of chance and I'm just overfitting to test set
df_submit = pd.DataFrame({'PassengerId': passengerIds, 'Survived': (merged3.mean(axis=1) > 0.49).astype(int)})
df_submit.to_csv(f'{submissions}/_blend053.csv', index=False)

In [156]:
corr_descending = merged.corr().mean().sort_values()
corr_descending_without_that_one_bad = corr_descending[1:]
corr_descending_without_that_one_bad

sub_14    0.255817
sub_11    0.262585
sub_10    0.271451
sub_25    0.300244
sub_20    0.300244
sub_8     0.342283
sub_19    0.355534
sub_18    0.396662
sub_24    0.420244
sub_13    0.452396
sub_6     0.455858
sub_23    0.457573
sub_21    0.468838
sub_9     0.490431
sub_5     0.493306
sub_16    0.500105
sub_3     0.518373
sub_22    0.529418
sub_7     0.530372
sub_0     0.530576
sub_12    0.538575
sub_4     0.543798
sub_2     0.550577
sub_17    0.551029
sub_1     0.554629
dtype: float64

In [162]:
selected_models = (corr_descending_without_that_one_bad < 0.4).index.values
merged4 = merged[selected_models]

In [168]:
# let it be majority voting this time, just curious though
(merged4.mean(axis=1) > 0.5).astype(int).mean()

0.38995215311004783

In [169]:
# LB: 0.78468, not bad
df_submit = pd.DataFrame({'PassengerId': passengerIds, 'Survived': (merged4.mean(axis=1) > 0.5).astype(int)})
df_submit.to_csv(f'{submissions}/_blend054.csv', index=False)